### Imports

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline

### Set de CUDA

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### HyperParams

In [4]:
input_size = 28
seq_len = 28
num_layers = 4
hidden_size = 256
num_classes = 10
lr = 0.001
batch_size = 32
epochs = 60

### Modelo RNN

In [5]:
class BRNN(nn.Module):
    
    def __init__(self,input_size,hidden_size,num_layers,num_classes):
        
        super(BRNN,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True, bidirectional=True) #las batches vienen en el primer eje por ser del dataloader
        self.fc = nn.Linear(hidden_size*2,num_classes) # el *2 es para las dos direcciones
        
    def forward(self,x):
        
        h0 = torch.zeros(self.num_layers*2,x.size(0),self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2,x.size(0),self.hidden_size).to(device)
        out,_ = self.lstm(x,(h0,c0))
        out = self.fc(out[:,-1,:])
        
        return out

### Load Data

In [6]:
train_dataset = datasets.MNIST(root="dataset/",train=True,transform=transforms.ToTensor(),download=True)

test_dataset = datasets.MNIST(root="dataset/",train=False,transform=transforms.ToTensor(),download=True)


In [7]:
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

### Inicializar RNN

In [8]:
model = BRNN(input_size,hidden_size,num_layers,num_classes).to(device)

### Criterion & Optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=lr)

### Entrenamiento red

In [10]:
for epoch in range(epochs):
    for batch_idx,(data,targets) in enumerate(train_loader):
        data = data.to(device).squeeze(1) # remover el 1 de la dimensión
        targets = targets.to(device)
        
        scores = model(data)
        loss = criterion(scores,targets)      
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        print(f"Entrenando epoch {epoch}")

Entrenando epoch 0
Entrenando epoch 10
Entrenando epoch 20
Entrenando epoch 30
Entrenando epoch 40
Entrenando epoch 50


### Check precisión

In [11]:
def check_accuracy(loader,model):
    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    if loader.dataset.train:
        print("Chequeando precisión de entrenamiento")
    else:
        print("Chequeando precisión de test")
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device).squeeze(1)
            y = y.to(device)
            
            scores = model(x)
            _,predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
            
        print(f"Tenemos {num_correct} / {num_samples} con precisión \
              {float(num_correct)/float(num_samples)*100:.2f}%")
        
    model.train()

In [12]:
check_accuracy(train_loader,model)

Chequeando precisión de entrenamiento
Tenemos 59954 / 60000 con precisión               99.92%


In [13]:
check_accuracy(test_loader,model)

Chequeando precisión de test
Tenemos 9903 / 10000 con precisión               99.03%
